In [6]:
import pandas as pd
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from matplotlib import style
import bs4 as bs
import pickle
import requests
import os
import datetime as dt
import numpy as np
import seaborn as sn
from collections import Counter
from sklearn import svm,neighbors, metrics
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc

style.use('ggplot')

First of all let's create 2 functions:
- Yhe first to retrive the Nasdaq tickers from internet and store the in a list
- The second for save in csv file, the stocks' information with respect to each ticket 

In [7]:
def save_nasdaq_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/Nasdaq-100')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        tickers.append(ticker)
        
    with open("nasdaq100_ticket.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

#save_nasdaq_tickers()

def get_data_from_yahoo(reload_sp500=False): # Here's where I'll just show a quick example of one way you could handle for 
                                             # whether or not to reload the S&P 500 list. If we ask it to, the program will 
                                             # re-pull the S&P 500 list, otherwise it will just use our pickle.
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("nasdaq100_ticket.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2016, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            #df = df.drop("Symbol", axis=1)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

            
get_data_from_yahoo()

Already have ATVI
Already have ADBE
Already have ADP
Already have ABNB
Already have ALGN
Already have GOOGL
Already have GOOG
Already have AMZN
Already have AMD
Already have AEP
Already have AMGN
Already have ADI
Already have ANSS
Already have AAPL
Already have AMAT
Already have ASML
Already have AZN
Already have TEAM
Already have ADSK
Already have BIDU
Already have BIIB
Already have BKNG
Already have AVGO
Already have CDNS
Already have CHTR
Already have CTAS
Already have CSCO
Already have CTSH
Already have CMCSA
Already have CEG
Already have CPRT
Already have COST
Already have CRWD
Already have CSX
Already have DDOG
Already have DXCM
Already have DOCU
Already have DLTR
Already have EBAY
Already have EA
Already have EXC
Already have FAST
Already have FISV
Already have FTNT
Already have GILD
Already have HON
Already have IDXX
Already have ILMN
Already have INTC
Already have INTU
Already have ISRG
Already have JD
Already have KDP
Already have KLAC
Already have KHC
Already have LRCX
Alrea

Now we want to create an unique dataset that contains adj_close data for all the 100 tickets

In [8]:
def compile_data():
    with open("nasdaq100_ticket.pickle","rb") as f:
            tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for ticker in tickers:
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close':ticker}, inplace=True) # we are renaming the adj close column with the ticket's name
        df.drop(['Open','High','Low','Close','Volume'], axis = 1,inplace=True) # '1' means tha we are dropping columns

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer') # outer is usefull for not loose information of all tickers  
                                                    # if for some tickers we don't have data for that day


    print(main_df.head())
    main_df.to_csv('nasdaq_joined_closes.csv')
    


compile_data()

                 ATVI       ADBE        ADP  ABNB       ALGN      GOOGL  \
Date                                                                      
2016-01-04  35.976723  91.970001  71.045609   NaN  64.269997  37.972000   
2016-01-05  35.517689  92.339996  71.218727   NaN  64.949997  38.076500   
2016-01-06  35.182983  91.019997  70.344421   NaN  64.870003  37.966499   
2016-01-07  34.685696  89.110001  68.197548   NaN  63.500000  37.049999   
2016-01-08  34.150146  87.849998  67.634834   NaN  64.529999  36.545502   

                 GOOG       AMZN   AMD        AEP  ...       TSLA        TXN  \
Date                                               ...                         
2016-01-04  37.091999  31.849501  2.77  46.350529  ...  14.894000  45.526722   
2016-01-05  37.129002  31.689501  2.75  46.731949  ...  14.895333  45.049168   
2016-01-06  37.181000  31.632500  2.51  46.906773  ...  14.602667  44.755928   
2016-01-07  36.319500  30.396999  2.28  46.366428  ...  14.376667  43.3148

Traceback (most recent call last):
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\39388\AppData\Local\Temp\ipykernel_13388\537361283.py", line 26, in <cell line: 26>
    compile_data()
  File "C:\Users\39388\AppData\Local\Temp\ipykernel_13388\537361283.py", line 22, in compile_data
    main_df.to_csv('nasdaq_joined_closes.csv')
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\pandas\util\_decorators.py", line 211, in wrapper
    return _deprecate_kwarg
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\pandas\core\generic.py", line 3721, in to_csv
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\pandas\util\_decorators.py", line 211, in wrapper
    return _deprecate_kwarg
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\pandas\io\formats\format.py", line 1170, in to_csv
    index_label

## Correlation Matrix

In [ ]:
def visualize_data():
    df = pd.read_csv('nasdaq_joined_closes.csv')

    df_corr = df.corr(numeric_only = 'TRUE')

    data1 = df_corr.values
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111) #111 is the size of the plot
    heatmap1 = ax1.pcolor(data1, cmap=plt.cm.RdYlGn)
    fig1.colorbar(heatmap1)

    ax1.set_xticks(np.arange(data1.shape[1]) + 0.5, minor=False)
    ax1.set_yticks(np.arange(data1.shape[0]) + 0.5, minor=False)
    ax1.invert_yaxis()
    ax1.xaxis.tick_top()
    column_labels = df_corr.columns
    row_labels = df_corr.index
    ax1.set_xticklabels(column_labels)
    ax1.set_yticklabels(row_labels)
    plt.xticks(rotation=90)
    heatmap1.set_clim(-1, 1)
    plt.tight_layout()
    plt.show()

visualize_data()

## Machine Learnign 

As featuresets we will take into account all company percent changes that day, and those will be our features. Our label will be whether or not Google(GOOG) rose more than x% within the next x days.

In [13]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('nasdaq_joined_closes.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)
    
    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker] #df[ticker].shift(-i) is the next day price, basically the row below
        df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    
    return df, tickers

This function create new columns, in which there are the % change of the price in the last n days, for the selected ticker.
So for each row we have, the closing price off the 100 companies that day, and the % change that the intersted firm has experienced in the following n days.

Es. GOOG_3d, means that in the next 3 day from that day, google has grown by tot%.

Of course for the last days we cant calculate those values, since we don't know the future values.

Now we ara going to write the function that creates our labels.

In [14]:
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [15]:
# just to check where we are
df,tickers = process_data_for_labels('GOOG')

print(df.tail())
print(df.shape)

                 ATVI        ADBE         ADP        ABNB        ALGN  \
Date                                                                    
2022-10-10  73.750000  285.720001  228.690002  110.989998  207.380005   
2022-10-11  73.550003  284.829987  226.270004  106.820000  208.449997   
2022-10-12  73.500000  286.149994  224.110001  112.349998  208.050003   
2022-10-13  73.120003  294.739990  229.899994  112.650002  212.309998   
2022-10-14  72.129997  287.940002  225.910004  109.160004  201.630005   

                GOOGL       GOOG        AMZN        AMD        AEP  ...  \
Date                                                                ...   
2022-10-10  97.860001  98.709999  113.669998  57.810001  84.449997  ...   
2022-10-11  97.180000  98.050003  112.209999  57.630001  84.870003  ...   
2022-10-12  97.559998  98.300003  112.900002  57.849998  81.739998  ...   
2022-10-13  99.059998  99.709999  112.529999  58.939999  84.779999  ...   
2022-10-14  96.559998  97.180000  106.

The following function will let us see the distributions of classes both in our dataset and in our algorithm's predictions. We dont want to feed highly imbalanced datasets to machine learning classifiers, and we also want to see if our classifier is predicting only one class.

In [16]:
def extract_featuresets(ticker):
    df, tickers = process_data_for_labels(ticker)
    
    # The map() function applies a given function to each item of an iterable (list, tuple etc.) and returns an iterator.
    # so in this case we are applying the buy_sell_hold function to all the columns that follows

    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,
                                               df['{}_1d'.format(ticker)],
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)] ))
    
    vals = df['{}_target'.format(ticker)].values.tolist() #is a list with all -1,0,1
    print(type(vals))
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))
    
    #clean the data
    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan) #if we pass from 0 to any value, the % change will be +inf 
    df.dropna(inplace=True)

    #right now our "features" are that day's prices for stocks. Just static numbers, really nothing telling at all. 
    # What we want company's percent change that day. 
    df_vals = df[[ticker for ticker in tickers]].pct_change() #by deafualt the shift for which is calculated the % change is 1

    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)
    
    X = df_vals.values
    y = df['{}_target'.format(ticker)].values
    return X, y, df

    
    



<span style="color:red"> PROBABILMENTE ORA COME ORA BUY_SELD_HOLD VIENE APPLICATO SOLO ALLA PRIMA COLLANNA CHE LI VIENE PASSAT,
E QUINDI IN QUESTO CASO AL PREZZO SUCCESSIVO DI UN GIORNO, NON SONO SICURO DI QUESTA COSA, ORA COME ORA PRENDILA COSì </span>



In [17]:
#just to check 
X,y,df = extract_featuresets('GOOG')


<class 'list'>
Data spread: Counter({'1': 771, '-1': 575, '0': 363})


Now we have created a function that gives back what we want as featuresets and labels, we can now use machine learning algorithms with the hope that they will learn to map relationships of existing price changes to future price changes for a company.

We'll try to make prediction using 3 differents ML algorithms: 
- Supported vector machine
- K Nearest Neighbors
- Random forest classifier

Combining them with a *voting classifiere*

Voting classifier simply aggregates the findings of each classifier passed into Voting Classifier and predicts the output class based on the highest majority of voting. The idea is instead of creating separate dedicated models and finding the accuracy for each them, we create a single model which trains by these models and predicts output based on their combined majority of voting for each output class.

In [35]:
# Create the confusion matrix
def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)

    # Create a dataframe with the confussion matrix values
    df_cm = pd.DataFrame(cm, range(cm.shape[0]),range(cm.shape[1]))
    #plt.figure(figsize = (10,7))
    # Plot the confussion matrix
    sn.set(font_scale=1.4) #for label size
    sn.heatmap(df_cm, annot=True,fmt='.0f',annot_kws={"size": 10})# font size
    plt.show()
    



def do_ml(ticker):
    X, y, df = extract_featuresets(ticker)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

    clf = VotingClassifier([('lsvc', svm.LinearSVC()),
                            ('knn', neighbors.KNeighborsClassifier()),
                            ('rfor', RandomForestClassifier())])

    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test) # with this we already perform prediction and compute measurements like accuracy, ecc..
    print('accuracy:', confidence)
    predictions = clf.predict(X_test) # we also do prediction in this way to see if our model is only classifying one class,
                                      # which is something that can easily happen.
    cm = confusion_matrix(y_test,predictions)
    print('predicted class counts:', Counter(predictions))
    print()
    print()
    return confidence,y_test,predictions


In [39]:
confidence,y_test,predictiosn = do_ml('AAPL')
plot_confusion_matrix(y_test,predictiosn)


<class 'list'>
Data spread: Counter({'1': 870, '-1': 578, '0': 261})
accuracy: 0.4883177570093458
predicted class counts: Counter({1: 387, -1: 41})
[[ 14   0 124]
 [  3   0  68]
 [ 24   0 195]]

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\39388\AppData\Local\Temp\ipykernel_13388\232327521.py", line 2, in <cell line: 2>
    plot_confusion_matrix(y_test,predictiosn)
  File "C:\Users\39388\AppData\Local\Temp\ipykernel_13388\1839283162.py", line 10, in plot_confusion_matrix
    sn.heatmap(df_cm, annot=True,fmt='.0f',annot_kws={"size": 10})# font size
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\seaborn\matrix.py", line 553, in heatmap
    ax = plt.gca()
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\matplotlib\pyplot.py", line 2227, in gca
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\matplotlib\pyplot.py", line 832, in gcf
  File "C:\Users\39388\anaconda3\envs\Finance\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    war